# How to use Regiopremise

Note that running this entire notebook will take you around 2 hours.

To be able to use Regiopremise, we will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5)
- a brightway project within which there is an ecoinvent database with the version 3.9.1 cut-off (only supported version so far)
- to download the trade database stored here: https://doi.org/10.5281/zenodo.13146833 Make sure to take the "treated" database where UN COMTRADE data was refined

In [1]:
path = 'path_to_your_regiopremise_project'

In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append(path)
import regioinvent

## Creation of a premise database with spatialized biosphere flows

The initialization of the Regioinvent class requires the following arguments:
- the path to the trade database
- the name you want to to give to the resulting regionalized ecoinvent database created
- the name of your brightway2 project
- the name of the premise database in your brightway2 project
- a boolean if you wish to regionalize elementary flows or not (True by default)
- a cutoff value to determine how "deep" regionalization should go. 0.99 cutoff leads to a ~185,000 processes database. Should not try to go higher than 0.99, code might bug as processes with new regions might show up, for which no mapping might have been done.

In [3]:
regio = regioinvent.Regioinvent(
    trade_database_path=f'{path}/trade_data_treated.db',
    regiopremise_database_name='Regiopremise', # not used while the rest of the notebook is WIP
    bw_project_name='ecoinvent3.9.1',
    premise_database_name='ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020',
    regionalized_elementary_flows=True,
    cutoff=0.99
)

2024-10-31 17:49:30,330 - Regioinvent - INFO - Extracting ecoinvent to wurst...


Getting activity data


100%|██████████| 32527/32527 [00:00<00:00, 70149.49it/s] 


Adding exchange data to activities


100%|██████████| 930582/930582 [00:47<00:00, 19671.32it/s]


Filling out exchange data


100%|██████████| 32527/32527 [00:05<00:00, 5904.93it/s] 
2024-10-31 17:50:38,748 - Regioinvent - INFO - Regionalizing the biosphere inputs of the original ecoinvent database...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:07


Title: Writing activities to SQLite3 database:
  Started: 10/31/2024 17:51:26
  Finished: 10/31/2024 17:52:33
  Total time elapsed: 00:01:07
  CPU %: 47.50
  Memory %: 28.14


AttributeError: `np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.

The initialization will first import everything related to regionalized elementary flows (if regionalized_elementary_flows=True), thus creating two new databases: a regionalized biosphere3 and a regionalized version of ecoinvent, as well as importing the regionalized version of the IMPACT World+ LCIA methodology. <br> The created regionalized version of ecoinvent also accounts for the creation of new water technosphere flows to correct inconsistencies in ecoinvent processes using water technosphere flows. <br>
It will then use the ```wurst``` Python library to extract your ecoinvent database from brightway into a list of dictionnary format which tremendously speed up operations made to the database.

## WIP: the rest of the notebook still requires adaptation to premise

Next, we extract and format the export data from the trade database

In [ ]:
regio.format_export_data()

We estimate the domestic production data based on the ratio of domestic vs export of the EXIOBASEv3.8.2 database 2019. This ratio is applied to self.export_data.

In [ ]:
regio.estimate_domestic_production()

We apply first order regionalization, that is for the 1870 traded products of the ecoinvent3.9.1 cut-off database, we determine the exporters of each product (through self.export_data) and then regionalize the following key inputs:
- electricity
- municipal solid waste
- heat

In [ ]:
regio.first_order_regionalization()

We then extract and format the import data from the trade database. The import data is combined with the domestic production to get total consumption data per product per country

In [ ]:
regio.format_import_data()

We determine consumption markets for each of the 1870 products.

In [ ]:
regio.create_consumption_markets()

We relink created consumption markets to regionalized processes across the regioinvent database.

In [ ]:
regio.second_order_regionalization()

We regionalize the elementary flows (if the corresponding boolean argument was selected)

In [ ]:
regio.regionalize_elem_flows()

Write regioinvent processes in the brightway2 SQL database.

In [ ]:
regio.write_regioinvent_to_database()

We reconnect ecoinvent processes to existing regioinvent processes, so that, e.g., electricity production, coal, CA uses the consumption market of coal for Canada that was created.

In [ ]:
regio.connect_ecoinvent_to_regioinvent()

Done!! You can now go on brightway2 or AB to perform your LCAs normally.